# 3. EM algorithm 

Generate samples from the mixture Gaussian Distribution 

In [6]:
import math
import copy
import numpy as np
 
def generate_data(num_samples, alpha, mu_list, sigma_list):    
    '''
    Generate tbe synthetic dataset from the mixture-of-Gaussian distribution
    Args:
        num_samples (positive integer)                  : number of samples
        alpha  (vector, num_classes)                    : prior probability vetor
        mu_list (list of length num_classes)            : the $i$-th element is the mean vector of $i$-th class
        sigma_list (list of length num_classes)         : the $i$-th element is the covariance matrix of $i$-th class
    Returns:
        X (matrix, num_samples * num_features)        : generated data
      
    '''
    
    num_components = len(mu_list)
    num_features = len(mu_list[0])
    X = np.zeros((num_samples, num_features))       
    # Generate random numbers in [0,1]
    random_numbers = np.random.random(num_samples)
    for i in range(num_samples):
        for j in range(num_components):
            if random_numbers[i] < sum(alpha[:j+1]):  
                X[i,:]  = np.random.multivariate_normal(mu_list[j], sigma_list[j], 1) 
                break
    return X

In [7]:
class EM_for_MG():
    '''
    This class is for using EM algorithm to estimate the parameters of mixture-of-Gaussian distribution.
    
    The class contains the parameters of EM iteration, including the number of classes $N$, the prior probabilities 
    $\alpha_i$, the mean vectors $\mu_i$ and covariance matrix $\Sigma_i$ for each class $i$.
    
    It also contains the functions for initializing the class, updating parameters in E-step and M-step, iterate over 
    the two steps until convergence.
    
    Attributes:
        num_classes (positive integer)           : the number of Gaussian components
        hat_alpha (list of length num_classes)   : the prior probability of each component
        hat_mu (list of length num_classes)      : the mean vector of each Gaussian component
        hat_sigma (list of length num_classes)   : the covariance matrix of each Gaussian component
        posterior_prob (matrix, num_samples * num_classes) : the posterior probability matrix and the $(j,i)$-th entry
            represents the posterior probability that the sample X[j,:] is from the $i$-th Gaussian component.
                                                   
        
    '''
    def __init__(self, num_classes=2, num_iteration=1000):
        '''
        Initialize the class for using EM algorithm to estimate the parameters in the Mixture-of-Gaussian model. 
        '''
        self.num_classes = num_classes
        self.num_iteration = num_iteration
        self.hat_alpha = []
        self.hat_mu = []
        self.hat_sigma = []
        self.posterior_prob = 0
        
    def fit(self, X):
        
        self.num_samples, self.num_features = X.shape
        ### Initialize parameters
        self.hat_alpha = [1/self.num_classes] * self.num_classes
        self.hat_mu = [np.min(X,axis=0) + (ell+1) / (self.num_classes+1) * (np.max(X, axis=0) - np.min(X, axis=0)) for ell in range(self.num_classes)]
        self.hat_sigma = [np.eye(self.num_features)*np.std(X,axis=0)] * self.num_classes
        ### Iteration begins
        previous_alpha = self.hat_alpha
        previous_mu = self.hat_mu
        for t in range(self.num_iteration):   
            ### E-step: Update posterior probability $\gamma_{ji}$
            self.E_step(X)    
            ### M-step: Update parameters $alpha, mu, sigma$
            self.M_step(X)    
            ### Judge whether the parameter estimations converge or not
            err_mu = np.mean(np.abs(np.array(previous_mu)-np.array(self.hat_mu)))     
            err_alpha = np.mean(np.abs(previous_alpha)-np.abs(self.hat_alpha))
            if (err_mu <= 0.001) and (err_alpha < 0.001):     
                print('Converged after {} iterations'.format(t+1))
                break
            else:
                previous_mu = self.hat_mu
                previous_alpha = self.hat_alpha
            ### print the result every 20 iterations
            if (t % 20 == 0):
                print('The number of iterations is:', t+1)
                print("The estimated mean vectors are:",self.hat_mu)
                print("The estimated prior probablilities are:",self.hat_alpha)
        return self.hat_alpha, self.hat_mu, self.hat_sigma
        
        
    def E_step(self, X):
        '''
        Calculate the posterior probablilty $\gamma_{ji}$ for each class $i$.
        '''
        self.posterior_prob = np.zeros((self.num_samples, self.num_classes))
        for j in range(self.num_samples):
            denom = 0
            for i in range(self.num_classes):
                denom += self.hat_alpha[i] * np.exp(-(X[j,:]-self.hat_mu[i]).reshape(1,-1)@np.linalg.inv(self.hat_sigma[i])@(X[j,:]-self.hat_mu[i]).reshape(-1,1)/2)[0,0]/np.sqrt(np.linalg.det(self.hat_sigma[i]))
            for i in range(self.num_classes):
                numer = np.exp(-(X[j,:]-self.hat_mu[i]).reshape(1,-1)@np.linalg.inv(self.hat_sigma[i])@(X[j,:]-self.hat_mu[i]).reshape(-1,1)/2)[0,0]/np.sqrt(np.linalg.det(self.hat_sigma[i]))   
                self.posterior_prob[j,i] = self.hat_alpha[i]*numer/denom      

    
    def M_step(self, X):
        '''
        Update the parameters $\alpha_i$, $\mu_i$ and $\Sigma_i$
        '''
        num_features = np.shape(X)[1]
        self.hat_mu, self.hat_alpha, self.hat_sigma = [], [], []
        for i in range(self.num_classes):
            denom=0   
            numer=0   
            for j in range(self.num_samples):
                numer += self.posterior_prob[j,i]*X[j,:]
                denom += self.posterior_prob[j,i]
            self.hat_mu.append(numer/denom)    
            self.hat_alpha.append(denom/self.num_samples)     
        for i in range(self.num_classes):
            cov_matrix = np.zeros((self.num_features,self.num_features))
            for j in range(self.num_samples):
                cov_matrix += self.posterior_prob[j,i] * np.dot((X[j,:] - self.hat_mu[i]).reshape(-1,1),(X[j,:] - self.hat_mu[i]).reshape(1,-1))
            self.hat_sigma.append(cov_matrix/np.sum(self.posterior_prob[:,i]))


In [8]:
num_samples = 1000         
num_components = 4            
alpha = [0.1,0.2,0.3,0.4]  
mu1 = [5,5]
mu2 = [10,15]
mu3 = [25,20]
mu4 = [45,30]
mu_list = [mu1, mu2, mu3, mu4]
sigma_list = [np.array([[10, 0], [0, 10]])]*4
dataset = generate_data(num_samples, alpha, mu_list, sigma_list) 
num_iteration = 1000
model = EM_for_MG(num_components, num_iteration)
hat_alpha, hat_mu, hat_sigma = model.fit(dataset)
print("The mean vectors converge to:", hat_mu)
print("The prior probablilities converge to:", hat_alpha)

The number of iterations is: 1
The estimated mean vectors are: [array([5.54740373, 6.39989141]), array([14.97425697, 16.1360408 ]), array([26.04041944, 21.09775304]), array([45.14208248, 30.00284033])]
The estimated prior probablilities are: [0.14781162761653793, 0.23755184520641426, 0.2139882530246035, 0.4006482741524441]
The number of iterations is: 21
The estimated mean vectors are: [array([4.86511192, 4.48628506]), array([10.04122609, 14.78472996]), array([24.72561463, 19.92091405]), array([45.08480489, 29.91543767])]
The estimated prior probablilities are: [0.10966891235083012, 0.18915836036094022, 0.29517846619104515, 0.4059942610971841]
Converged after 36 iterations
The mean vectors converge to: [array([4.71381662, 4.2074898 ]), array([ 9.96878205, 14.62347069]), array([24.72846409, 19.9208832 ]), array([45.08480369, 29.91543702])]
The prior probablilities converge to: [0.10374668320673734, 0.19519354885398635, 0.2950654422147265, 0.4059943257245499]
